# Remove unavailable videos and change names (uncommented)

In [9]:
import json
import os
import cv2
from pathlib import Path

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
json_file = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/WLASL_v0.3.json"

with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

video_labels = {}
for entry in data:
    gloss = entry["gloss"]
    for instance in entry["instances"]:
        video_id = str(instance["video_id"])  
        if video_id not in video_labels:
            video_labels[video_id] = []
        video_labels[video_id].append(gloss)

video_folder = Path(video_folder)

if not video_folder.exists():
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = list(video_folder.glob("*"))


deleted_videos = []
renamed_videos = []

def is_video_valid(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return False  
    ret, _ = cap.read()
    cap.release()
    return ret 

for video_file in video_files:
    if not video_file.is_file():
        continue

    video_id = video_file.stem 

    if os.path.getsize(video_file) == 0 or not is_video_valid(video_file):
        deleted_videos.append(video_file.name)
        os.remove(video_file)
        continue

    # labels = video_labels.get(video_id, [])
    # if labels:
    #     new_name = f"{video_id}_{'_'.join(labels)}{video_file.suffix}"
    #     new_path = video_file.parent / new_name
    #     os.rename(video_file, new_path)
    #     renamed_videos.append((video_file.name, new_name))

print("Gelöschte Videos:", deleted_videos)
print("Umbenannte Videos:", renamed_videos)


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d2040d9380] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d2040d7b00] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d2046f7600] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d2046f7600] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d2040e7700] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d2040e7700] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204296ec0] moov atom no

Gelöschte Videos: ['58968_tornado.mp4', '45913_radio.mp4', '01513_agenda.mp4', '68020_chat.mp4', '18218_earring.mp4', '70346_today.mp4', '20310_expand.mp4', '69008_nephew.mp4', '55558_subway.mp4', '48345_robot.mp4', '20384_experience.mp4', '50621_seven.mp4', '67413_birthday.mp4', '20137_exchange.mp4', '16392_dirt.mp4', '67224_soda.mp4', '67745_hat.mp4', '70040_everyday.mp4', '57979_think.mp4', '68185_win.mp4', '68021_class.mp4', '63662_woman.mp4', '67496_china.mp4', '45871_race.mp4', '67103_print.mp4', '68177_walk.mp4', '23469_french fries.mp4', '67533_counselor.mp4', '13180_cookie.mp4', '57542_testify.mp4', '01721_airplane.mp4', '06602_blend.mp4', '68132_pink.mp4', '32189_laptop.mp4', '37371_music.mp4', '68066_hard of hearing.mp4', '51779_since.mp4', '62187_wall.mp4', '08439_butter.mp4', '31047_jesus.mp4', '68752_bike.mp4', '41709_pear.mp4', '23002_fork.mp4', '25693_green.mp4', '14276_cup.mp4', '42546_physician.mp4', '07860_brochure.mp4', '68860_hide.mp4', '31216_journey.mp4', '49003_

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204d1d440] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204d1d440] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204d1d440] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204d1d440] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d204d1d440] moov atom not found


# Add frame count to video names

In [11]:
import cv2
from pathlib import Path

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

def get_frame_count(video_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None 
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

renamed_videos = []

for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    video_id, ext = os.path.splitext(video_file) 
    
    frame_count = get_frame_count(video_path)
    
    if frame_count is not None:
        new_name = f"{video_id}_{frame_count}{ext}"
        new_path = os.path.join(video_folder, new_name)
        os.rename(video_path, new_path)
        renamed_videos.append((video_file, new_name))

print("Umbenannte Videos mit Frames:", renamed_videos)

Umbenannte Videos mit Frames: [('62275_war.mp4', '62275_war_71.mp4'), ('58013_third.mp4', '58013_third_30.mp4'), ('17959_due.mp4', '17959_due_86.mp4'), ('39080_nun.mp4', '39080_nun_48.mp4'), ('54007_spoon.mp4', '54007_spoon_43.mp4'), ('62619_wear.mp4', '62619_wear_53.mp4'), ('24956_go.mp4', '24956_go_46.mp4'), ('65365_clean.mp4', '65365_clean_73.mp4'), ('25052_goodbye.mp4', '25052_goodbye_74.mp4'), ('53621_speak.mp4', '53621_speak_88.mp4'), ('56198_swallow.mp4', '56198_swallow_40.mp4'), ('61986_vomit.mp4', '61986_vomit_31.mp4'), ('15810_dessert.mp4', '15810_dessert_40.mp4'), ('62245_want.mp4', '62245_want_72.mp4'), ('59497_travel.mp4', '59497_travel_55.mp4'), ('23664_front.mp4', '23664_front_63.mp4'), ('14773_dawn.mp4', '14773_dawn_75.mp4'), ('23953_future.mp4', '23953_future_45.mp4'), ('10625_choose.mp4', '10625_choose_69.mp4'), ('21993_firefighter.mp4', '21993_firefighter_98.mp4'), ('05238_basketball.mp4', '05238_basketball_42.mp4'), ('36454_mix.mp4', '36454_mix_62.mp4'), ('69534_who

# Informations about videos

In [4]:
import os
from collections import Counter

video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

if not os.path.exists(video_folder):
    raise FileNotFoundError(f"Der Ordner {video_folder} wurde nicht gefunden.")

video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

label_counter = Counter()

for video_file in video_files:
    filename, _ = os.path.splitext(video_file) 
    parts = filename.split("_") 

    if len(parts) == 3:  
        label = parts[1]  
        label_counter[label] += 1

print(f"Anzahl der verschiedenen Labels: {len(label_counter)}")
print("Top 10 häufigste Labels:")
for label, count in label_counter.most_common(10):
    print(f"{label}: {count}")


Anzahl der verschiedenen Labels: 168
Top 10 häufigste Labels:
thin: 15
go: 14
help: 13
bowling: 13
before: 13
computer: 13
cool: 13
drink: 12
tall: 12
trade: 12


In [3]:
# Gruppen nach Anzahl der Videos pro Label
from collections import Counter

label_distribution = Counter(label_counter.values())

print("Anzahl der Labels mit bestimmter Anzahl von Videos:")
for num_videos, num_labels in sorted(label_distribution.items()):
    print(f"{num_labels} Labels haben {num_videos} Videos")

Anzahl der Labels mit bestimmter Anzahl von Videos:
158 Labels haben 7 Videos
83 Labels haben 8 Videos
46 Labels haben 9 Videos
15 Labels haben 10 Videos
11 Labels haben 11 Videos
6 Labels haben 12 Videos
5 Labels haben 13 Videos
1 Labels haben 14 Videos
1 Labels haben 15 Videos


# Delete videos with less than min_videos_per_label

In [2]:
import os
from collections import Counter

# Verzeichnis mit den Videos
video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"

# Liste aller Videos abrufen
video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]

# Labels extrahieren und zählen
label_counter = Counter()

for video_file in video_files:
    filename, _ = os.path.splitext(video_file)  # Entfernt Dateiendung
    parts = filename.split("_")  # Trennt nach "_"
    
    if len(parts) >= 3:  # Sicherstellen, dass das Format korrekt ist
        label = parts[1]  # Mittleres Element ist das Label
        label_counter[label] += 1

# Mindestanzahl an Videos pro Label
min_videos_per_label = 8

# Dateien mit zu wenigen Videos löschen
deleted_count = 0

for video_file in video_files:
    filename, _ = os.path.splitext(video_file)
    parts = filename.split("_")

    if len(parts) >= 3:
        label = parts[1]
        if label_counter[label] < min_videos_per_label:
            os.remove(os.path.join(video_folder, video_file))
            deleted_count += 1

print(f"{deleted_count} Videos wurden gelöscht, da ihr Label weniger als {min_videos_per_label} Videos hatte.")

1106 Videos wurden gelöscht, da ihr Label weniger als 8 Videos hatte.


# Print videos with less than 40 min_videos_per_label

In [11]:
import os
from collections import Counter

# Verzeichnisse mit den Videos
raw_video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_video_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Labels zählen
label_counter = Counter()

for folder in [raw_video_folder, augmented_video_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        parts = video_file.split("_")
        if len(parts) >= 4:  # Sicherstellen, dass der Name korrekt ist
            label = parts[3]  # Das dritte Element ist das Label
            label_counter[label] += 1

# Sortiere die Labels nach Anzahl der Videos (absteigend)
sorted_labels = sorted(label_counter.items(), key=lambda x: x[1], reverse=True)

# Anzahl der verfügbaren Labels
print(f"Anzahl der verbleibenden Labels: {len(sorted_labels)}")

# Top 10 häufigste Labels ausgeben
print("\nTop 10 häufigste Labels:")
for label, count in sorted_labels[:10]:
    print(f"{label}: {count}")

# Alle Labels mit Anzahl der Videos ausgeben
print("\nAnzahl der Videos pro Label:")
for label, count in sorted_labels:
    print(f"{label}: {count}")


Anzahl der verbleibenden Labels: 168

Top 10 häufigste Labels:
thin: 90
go: 84
bowling: 78
computer: 78
help: 78
before: 78
cool: 78
trade: 72
bed: 72
thanksgiving: 72

Anzahl der Videos pro Label:
thin: 90
go: 84
bowling: 78
computer: 78
help: 78
before: 78
cool: 78
trade: 72
bed: 72
thanksgiving: 72
drink: 72
accident: 72
tall: 72
change: 66
dark: 66
pizza: 66
cold: 66
man: 66
corn: 66
who: 66
shirt: 66
cousin: 66
short: 66
candy: 66
deaf: 60
basketball: 60
woman: 60
thursday: 60
apple: 60
far: 60
family: 60
later: 60
leave: 60
call: 60
laugh: 60
dog: 60
delay: 60
champion: 60
yes: 60
letter: 54
graduate: 54
white: 54
fish: 54
yesterday: 54
no: 54
many: 54
balance: 54
hot: 54
play: 54
delicious: 54
mother: 54
cheat: 54
black: 54
why: 54
sandwich: 54
what: 54
study: 54
good: 54
ready: 54
work: 54
chat: 54
give: 54
brother: 54
language: 54
move: 54
secretary: 54
full: 54
last: 54
write: 54
score: 54
toast: 54
carrot: 54
year: 54
convince: 54
argue: 54
daughter: 54
check: 54
fat: 54
bad

# Calculate maximum number of frames

In [ ]:
import os
import torch

# Verzeichnisse mit Videos
original_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/raw_videos"
augmented_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_augmented"

# Bestimme die maximale Anzahl an Frames über beide Ordner
max_frames = 0

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        parts = video_file.split("_")
        if len(parts) >= 3:
            frame_count = int(parts[-1].replace(".mp4", ""))  # Frame-Anzahl extrahieren
            max_frames = max(max_frames, frame_count)

print(f"Maximale Anzahl an Frames (über alle Videos): {max_frames}")

# Reduce number of videos per label

In [ ]:
import random
from collections import Counter

# Zähle, wie viele Videos pro Label existieren (original & augmented)
label_counter = Counter()

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        parts = video_file.split("_")
        if len(parts) >= 3:
            label = parts[1]  # Mittleres Element ist das Label
            label_counter[label] += 1

# Finde das Minimum an Videos pro Label
min_videos_per_label = min(label_counter.values())
print(f"Jedes Label wird auf {min_videos_per_label} Videos reduziert.")

# Entferne überschüssige augmentierte Videos zufällig
for label, count in label_counter.items():
    if count > min_videos_per_label:
        # Finde alle Videos dieses Labels NUR in den augmentierten Videos
        label_videos = [f for f in os.listdir(augmented_folder) if f.split("_")[1] == label]
        
        # Wähle zufällig die zu löschenden Videos
        remove_videos = random.sample(label_videos, count - min_videos_per_label)
        
        # Lösche die Videos
        for video_file in remove_videos:
            os.remove(os.path.join(augmented_folder, video_file))

print("Alle Labels haben jetzt die gleiche Anzahl an Videos!")

# Normalize frames

In [ ]:
import cv2
import torch
import numpy as np

processed_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_processed"
os.makedirs(processed_folder, exist_ok=True)

# Funktion zur Extraktion von Frames als Torch-Tensoren
def extract_frames(video_path, device="cuda"):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))  # Falls nötig, Frames auf gleiche Größe bringen
        frame = torch.tensor(frame, dtype=torch.float32, device=device).permute(2, 0, 1)  # [H, W, C] → [C, H, W]
        frames.append(frame)

    cap.release()
    return torch.stack(frames) if frames else torch.zeros((1, 3, 224, 224), dtype=torch.float32, device=device)  # Falls leer, Dummy-Frame

# Funktion zum Padding mit GPU-Unterstützung
def pad_frames(frames, target_length, device="cuda"):
    num_frames = frames.shape[0]
    
    if num_frames < target_length:
        padding = torch.zeros((target_length - num_frames, 3, 224, 224), dtype=torch.float32, device=device)
        return torch.cat((frames.to(device), padding), dim=0)
    else:
        return frames[:target_length].to(device)  # Falls zu lang, einfach abschneiden

# Verarbeitung aller Videos (Original + Augmented)
device = "cuda" if torch.cuda.is_available() else "cpu"

for folder in [original_folder, augmented_folder]:
    for video_file in os.listdir(folder):
        if not video_file.endswith(".mp4"):
            continue

        video_path = os.path.join(folder, video_file)

        # Frames extrahieren und auf GPU laden
        frames = extract_frames(video_path, device=device)

        # Padding / Kürzen auf `max_frames`
        padded_frames = pad_frames(frames, max_frames, device=device)

        # Speichern als `.npy`-Datei (wieder auf CPU konvertieren)
        np.save(os.path.join(processed_folder, video_file.replace(".mp4", ".npy")), padded_frames.cpu().numpy())

print("GPU-basierte Normalisierung abgeschlossen!")

# Extract keypoints with Mediapipe Holistics

In [ ]:
import os
import torch
import numpy as np
import cv2
import mediapipe as mp

# Verzeichnisse
processed_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/videos_processed"
keypoints_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints_gpu"
os.makedirs(keypoints_folder, exist_ok=True)

# CUDA-Gerät initialisieren
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Verwende Gerät: {device}")

# MediaPipe Holistic initialisieren (läuft nur auf CPU)
mp_holistic = mp.solutions.holistic.Holistic(static_image_mode=False, model_complexity=1, smooth_landmarks=True, enable_segmentation=False)

# Funktion zur Extraktion von Keypoints (Pose + Hände)
def extract_keypoints(results):
    pose = torch.tensor([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark], dtype=torch.float32, device=device).flatten() if results.pose_landmarks else torch.zeros(33*3, dtype=torch.float32, device=device)
    left_hand = torch.tensor([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark], dtype=torch.float32, device=device).flatten() if results.left_hand_landmarks else torch.zeros(21*3, dtype=torch.float32, device=device)
    right_hand = torch.tensor([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark], dtype=torch.float32, device=device).flatten() if results.right_hand_landmarks else torch.zeros(21*3, dtype=torch.float32, device=device)
    
    return torch.cat([pose, left_hand, right_hand])  # Gesamt: 99 Keypoints pro Frame

# Funktion zur Verarbeitung eines Videos mit GPU-Unterstützung
def process_video_on_gpu(video_path):
    frames = torch.tensor(np.load(video_path), dtype=torch.uint8, device=device)  # Lade das Video direkt als Tensor auf die GPU
    keypoints_seq = []

    for frame in frames:
        # MediaPipe benötigt NumPy-Format -> Verschiebe Frame zurück auf CPU
        frame_np = frame.cpu().numpy()

        # MediaPipe Holistic auf CPU ausführen
        results = mp_holistic.process(frame_np)

        # Keypoints extrahieren und auf GPU speichern
        keypoints = extract_keypoints(results)
        keypoints_seq.append(keypoints)

    return torch.stack(keypoints_seq) if keypoints_seq else torch.zeros((1, 99), dtype=torch.float32, device=device)  # Falls Video fehlerhaft ist

# CUDA-Stream für parallele Verarbeitung
stream = torch.cuda.Stream()

# Alle Videos mit GPU-Support verarbeiten
with torch.cuda.stream(stream):
    for video_file in os.listdir(processed_folder):
        if not video_file.endswith(".npy"):
            continue
        
        video_path = os.path.join(processed_folder, video_file)

        # Verarbeite Video auf der GPU
        keypoints = process_video_on_gpu(video_path)

        # Speichere als PyTorch-Tensor (.pt statt .npy)
        torch.save(keypoints, os.path.join(keypoints_folder, video_file.replace(".npy", ".pt")))

print("GPU-optimierte Keypoint-Extraktion abgeschlossen!")